In [6]:
import models

In [7]:
name = 'INFLACAO'
file_name = 'INFLACAO_2022-12-19-16-38-27-184580.csv'
directory = f'DataLakeRiscoECompliance/PrevisionData/Variables/{name}/AI'

In [8]:
## Upload a file
multithread.ADLUploader(adlsFileSystemClient, lpath='output/' + file_name, rpath=f'{directory}/{file_name}', nthreads=64, overwrite=True, buffersize=4194304, blocksize=4194304)

<ADL Upload: output/INFLACAO_2022-12-19-16-38-27-184580.csv -> DataLakeRiscoECompliance\PrevisionData\Variables\INFLACAO\AI\INFLACAO_2022-12-19-16-38-27-184580.csv ({'finished': 1})>